# MACHINE LEARNING

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
%matplotlib inline

In [ ]:
# Load the data
american_politicians_df = pd.read_parquet('../data/american_politicians/parquet/', engine='pyarrow')
# Parse the date column to datetime
american_politicians_df['created_at'] = pd.to_datetime(american_politicians_df['created_at'])
display(american_politicians_df.head())

In [ ]:
def plot_scatterplots(df, columns, x_label):
    """Plot scatterplots of the given columns against the x_label.
    
    Parameters
    ----------
    df : pandas.DataFrame
        The DataFrame to plot.
    columns : list
        The columns to plot.
    x_label : str
        The column to plot on the x-axis.
        """
    nb_rows = len(columns)
    height = 4 * nb_rows
    fig, axes = plt.subplots(nb_rows, 2, figsize=(16, height))
    colors = ['royalblue', 'dodgerblue', 'cornflowerblue', 'skyblue', 'lightsteelblue', 'lightblue', 'lightskyblue', 'powderblue']
    i = 0 

    for column in columns:
        x_label_fmt = x_label.replace('_', ' ').title()
        y_label_fmt = column.replace('_', ' ').title()

        axes = axes.flatten()
        ax1 = axes[i//2*2]
        sns.scatterplot(x=df[x_label], y=df[column], color=colors[i//2], alpha=0.5, ax=ax1)
        sns.regplot(x=df[x_label], y=df[column], color=colors[i//2], scatter=True, ax=ax1)
        ax1.set_xlabel(x_label_fmt, fontsize=8)
        ax1.set_ylabel(y_label_fmt, fontsize=8)
        ax1.set_title(f'{y_label_fmt} as a Function of {x_label_fmt} in Linear Scale', fontsize=9)
        ax1.set_xscale('linear')
        ax1.set_yscale('linear')
        
        ax2 = axes[i//2*2+(i+1)%2]
        sns.scatterplot(x=df[x_label], y=df[column], color=colors[i//2], alpha=0.5, ax=ax2)
        ax2.set_xlabel(x_label_fmt, fontsize=8)
        ax2.set_ylabel(y_label_fmt, fontsize=8)
        ax2.set_title(f'{y_label_fmt} as a Function of {x_label_fmt} in Log Scale', fontsize=9)
        ax2.set_xscale('log')
        ax2.set_yscale('log')
        
        i += 2
    plt.tight_layout()
    plt.show()

In [ ]:
# Plot the scatterplots of the engagement counts against impression count
columns = ['retweet_count', 'reply_count', 'like_count', 'quote_count']
plot_scatterplots(american_politicians_df, columns, 'impression_count')

In [ ]:
# Plot the scatterplots of the impression counts against follower counts
columns = ['impression_count']
plot_scatterplots(american_politicians_df, columns, 'followers_count')

### GET FEATURES FROM DATAFRAME

Tweet info:
- Tweet length  ✅
- Time of the tweet (morning, afternoon, night) ✅
- Sentiment of the tweet (score computed by model) ✅
- Number of hashtags ✅
- Number of mentions ✅
- Number of url's ✅
- Media type (video, image, text,..) => attention c un tableau
- (To clarify: location) 🚧

User info:
- Verified ✅
- Profile creation date ✅
- (To clarify: Tweet frequency) 🚧

## Extracting features from raw data

In [ ]:
from pyspark.sql import SparkSession
# Import spark and open json file
spark = SparkSession.builder.getOrCreate()
df = spark.read.json('../data/american_politicians/tweets.jsonl')
json_rdd = df.rdd
df.printSchema()
spark

In [ ]:
import vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from datetime import datetime

# Function to get the sentiment of a tweet
def analyse_sentiment(x):
    analyzer = SentimentIntensityAnalyzer()
    vs = analyzer.polarity_scores(x)
    return vs['compound']

# 
def add_key_value(x, key, value):
    x[key] = value
    return x


def keep_medias_only(x):
    urls = x['tweet_urls']
    if not urls:
        return []
    media_urls = [url['media_key'] for url in urls if 'media_key' in url and url['media_key']]
    return media_urls

def get_number_medias(x):
    return len(x['tweet_media_keys'])

def get_number_external_urls(x):
    return (len(x['tweet_urls']) if x['tweet_urls'] else 0) - x['tweet_medias_count']

def get_period_of_day(x):
    hour = datetime.strptime(x['tweet_date'],"%Y-%m-%dT%H:%M:%S.%fZ").hour
    if hour >= 6 and hour < 12:
        return 'morning'
    elif hour >= 12 and hour < 18:
        return 'afternoon'
    else:
        return 'night'

In [ ]:
json_rdd_data_fields =json_rdd.flatMap(lambda x: x['data']) \
.map(lambda x : {
    'tweet_text': x['text'],
    'tweet_date': x['created_at'],
    'tweet_hashtags': x['entities']['hashtags'],
    'tweet_mentions': x['entities']['mentions'],
    'tweet_urls': x['entities']['urls'], 
    'user_id': x['author_id'],
})

# adding sentiment analysis on the tweet text using vader to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'sentiment', analyse_sentiment(x['tweet_text'])))

# adding the tweet length to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'tweet_length', len(x['tweet_text'])))

# adding the number of hashtags to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'hashtags_count', len(x['tweet_hashtags'])if x['tweet_hashtags'] else 0))

# adding the number of mentions to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'mentions_count', len(x['tweet_mentions'])if x['tweet_mentions'] else 0))

# adding the media url's only to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'tweet_media_keys', keep_medias_only(x)))

# adding the number of medias to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'tweet_medias_count', get_number_medias(x)))

# adding the number of external urls to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'tweet_external_urls_count', get_number_external_urls(x)))

# adding the period of the day to the data
json_rdd_data_fields = json_rdd_data_fields.map(lambda x : add_key_value(x, 'tweet_period', get_period_of_day(x)))
display(json_rdd_data_fields.take(1))

# json_rdd_data_fields_df = json_rdd_data_fields.toDF(['tweet_text', 'tweet_date', 'tweet_hashtags', 'tweet_mentions', 'tweet_urls', 'user_id', 'sentiment', 'tweet_length', 'hashtags_count', 'mentions_count', 'tweet_media_keys', 'tweet_medias_count', 'tweet_external_urls_count', 'tweet_period'])
# json_rdd_data_fields_df.show(5)

In [ ]:
json_rdd_includes_fields =json_rdd \
.map(lambda x : {
    'user_profile_creation_date': x['includes']['users'][0]['created_at'],
    'user_verified': x['includes']['users'][0]['verified'],
    'user_creation_date': x['includes']['users'][0]['created_at'],
    'user_id': x['includes']['users'][0]['id'],
})

json_rdd_includes_fields.take(10)

In [ ]:
json_rdd_media = json_rdd.flatMap(lambda x: x['includes']['media']).map(lambda x: [x["media_key"], x["type"], x["url"]] ).toDF(["media_key", "type", "url"])
json_rdd_media.show(10)